In [1]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'
final_result = '/content/drive/MyDrive/my_project/my_project1/result/data'
final_img = '/content/drive/MyDrive/my_project/my_project1/result/img'
# rawdata_path = 'rawdata/'
# data_path = 'data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# # 1. 나눔 폰트 다운
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
from mpl_toolkits.axes_grid1 import make_axes_locatable

import geopandas as gpd

import statsmodels.api as sm
import re
from io import StringIO

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [4]:
year_list = ['2022', '2023', '2024']

for year in year_list:
    globals()[f'temp_{year}'] = pd.read_csv(f'{data_path}/final/final_{year}.csv')

    globals()[f'temp_{year}'][f'solo_20s_{year}'] = globals()[f'temp_{year}'].apply(lambda x: (x[f'solo_20_{year}'] + x[f'solo_25_{year}']) / 2, axis = 1)
    globals()[f'temp_{year}'][f'solo_all_ratio'] = globals()[f'temp_{year}'].apply(lambda x: x[f'solo_all_{year}'] / x[f'total_all_{year}'] * 100, axis=1)
    globals()[f'temp_{year}'][f'solo_20s_ratio'] = globals()[f'temp_{year}'].apply(lambda x: x[f'solo_20s_{year}'] / x[f'solo_all_{year}'] * 100, axis=1)



In [5]:
reg_col = ['ADM_CD', 'solo_all_ratio', 'solo_20s_ratio',
           '네일숍', '미용실', '애완동물',
           '분식전문점', '한식음식점', '양식음식점', '일식음식점', '중식음식점', '치킨전문점',
           '커피-음료', '패스트푸드점', '제과점', '호프-간이주점',
           '유통업체수', '대형마트', '쇼핑업체수', '편의점', '슈퍼마켓',
           '지가변동률', '지가',
           '사업체수', '종사자수', '금융기관',
           '대학교', '기타대학', '면적', '월평균소득',
           '총상주인구', '20대상주인구', '총직장인구', '20대직장인구']

In [6]:
df_2022 = temp_2022.copy()
df_2022.head()

,ADM_CD,total_all_2022,solo_all_2022,total_20_2022,solo_20_2022,total_25_2022,solo_25_2022,네일숍_2022,미용실_2022,분식전문점_2022,...,총매출금액_중식음식점_2022,총매출금액_치킨전문점_2022,총매출금액_커피-음료_2022,총매출금액_패스트푸드점_2022,총매출금액_편의점_2022,총매출금액_한식음식점_2022,총매출금액_호프-간이주점_2022,solo_20s_2022,solo_all_ratio,solo_20s_ratio
0,11010530,7848.0,1854.0,582.0,183.0,687.0,220.0,13.0,30.0,127.0,...,2.067096e+09,151123891.0,8.715381e+09,749006078.0,4.259033e+09,2.279076e+10,1.627539e+09,201.5,23.623853,10.868393
1,11010540,2384.0,831.0,156.0,78.0,172.0,90.0,1.0,0.0,17.0,...,0.000000e+00,0.0,4.847156e+09,7462031.0,4.966037e+08,4.833707e+09,1.334099e+08,84.0,34.857383,10.108303
2,11010550,9116.0,1530.0,1147.0,201.0,841.0,154.0,0.0,10.0,14.0,...,4.502295e+07,27441593.0,4.592954e+08,107221827.0,7.261664e+08,5.864678e+08,1.911793e+07,177.5,16.783677,11.601307
3,11010560,15369.0,1604.0,1269.0,132.0,1173.0,127.0,2.0,16.0,15.0,...,2.886612e+08,0.0,5.489234e+08,0.0,4.087258e+08,2.379329e+09,1.762930e+08,129.5,10.436593,8.073566
4,11030740,18531.0,5332.0,1138.0,439.0,1829.0,748.0,11.0,46.0,27.0,...,9.877729e+08,770611211.0,2.705864e+09,228290638.0,3.617802e+09,1.480354e+10,2.165285e+09,593.5,28.773407,11.130908


In [7]:
columns_2022 = [col.replace('_2022', '') for col in df_2022.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2022.columns = columns_2024
df_2022.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', '네일숍', '미용실', '분식전문점', '애완동물', '한식음식점', '양식음식점', '일식음식점',
       '중식음식점', '치킨전문점', '커피-음료', '패스트푸드점', '제과점', '편의점', '호프-간이주점', '지가변동률',
       '지가', '사업체수', '종사자수', '금융기관', '유통업체수', '대형마트', '쇼핑업체수', '대학교', '기타대학',
       '면적', '월평균소득', '총상주인구', '20대상주인구', '총직장인구', '20대직장인구', '슈퍼마켓',
       '20대매출건수_네일숍', '20대매출건수_미용실', '20대매출건수_분식전문점', '20대매출건수_애완동물',
       '20대매출건수_양식음식점', '20대매출건수_일식음식점', '20대매출건수_제과점', '20대매출건수_중식음식점',
       '20대매출건수_치킨전문점', '20대매출건수_커피-음료', '20대매출건수_패스트푸드점', '20대매출건수_편의점',
       '20대매출건수_한식음식점', '20대매출건수_호프-간이주점', '20대매출금액_네일숍', '20대매출금액_미용실',
       '20대매출금액_분식전문점', '20대매출금액_애완동물', '20대매출금액_양식음식점', '20대매출금액_일식음식점',
       '20대매출금액_제과점', '20대매출금액_중식음식점', '20대매출금액_치킨전문점', '20대매출금액_커피-음료',
       '20대매출금액_패스트푸드점', '20대매출금액_편의점', '20대매출금액_한식음식점', '20대매출금액_호프-간이주점',
       '총매출건수_네일숍', '총매출건수_미용실', '총매출건수_분식전문점', '총매출건수_애완동물', '총매출건수_양식음식점',
       '총매출건수_일식음식점', '총매출건수_제과

In [8]:
reg_2022 = df_2022[reg_col]
reg_2022

,ADM_CD,solo_all_ratio,solo_20s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,일식음식점,...,종사자수,금융기관,대학교,기타대학,면적,월평균소득,총상주인구,20대상주인구,총직장인구,20대직장인구
0,11010530,23.623853,10.868393,13.0,30.0,10.0,127.0,413.0,183.0,55.0,...,56046,23,0.0,0.0,1.23,5007768.0,9524.0,1238.0,47822.0,6303.0
1,11010540,34.857383,10.108303,1.0,0.0,0.0,17.0,73.0,85.0,7.0,...,5118,3,0.0,0.0,1.49,3442534.0,2635.0,320.0,5862.0,726.0
2,11010550,16.783677,11.601307,0.0,10.0,6.0,14.0,41.0,37.0,4.0,...,4140,1,1.0,0.0,2.27,3647449.0,10015.0,1340.0,4115.0,695.0
3,11010560,10.436593,8.073566,2.0,16.0,3.0,15.0,56.0,19.0,5.0,...,4294,2,0.0,0.0,8.87,4504006.0,18390.0,2450.0,23179.0,2878.0
4,11030740,28.773407,11.130908,11.0,46.0,14.0,27.0,197.0,210.0,43.0,...,20440,11,0.0,0.0,3.01,4381088.0,21151.0,2782.0,41934.0,6886.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,28.133635,8.928571,0.0,8.0,1.0,21.0,67.0,44.0,7.0,...,15449,4,0.0,0.0,0.54,3479766.0,4325.0,493.0,11728.0,1263.0
418,11010610,49.126157,12.002990,11.0,35.0,11.0,240.0,1173.0,316.0,108.0,...,106826,42,1.0,0.0,2.35,3245251.0,7242.0,878.0,97451.0,19755.0
419,11220630,13.926386,11.276252,53.0,227.0,59.0,103.0,525.0,105.0,69.0,...,16795,0,0.0,0.0,1.93,3864301.0,20247.0,2861.0,16805.0,2831.0
420,11090700,24.407020,12.764194,52.0,309.0,61.0,112.0,654.0,30.0,50.0,...,7190,0,0.0,1.0,0.93,2318870.0,22217.0,3868.0,1850.0,199.0


In [9]:
df_2023 = temp_2023.copy()
df_2023.head()

,ADM_CD,total_all_2023,solo_all_2023,total_20_2023,solo_20_2023,total_25_2023,solo_25_2023,네일숍_2023,미용실_2023,분식전문점_2023,...,총매출금액_중식음식점_2023,총매출금액_치킨전문점_2023,총매출금액_커피-음료_2023,총매출금액_패스트푸드점_2023,총매출금액_편의점_2023,총매출금액_한식음식점_2023,총매출금액_호프-간이주점_2023,solo_20s_2023,solo_all_ratio,solo_20s_ratio
0,11010530,7845.0,1869.0,583.0,172.0,686.0,241.0,13.0,32.0,124.0,...,2.749773e+09,673323360.0,1.187346e+10,902523398.0,5.084405e+09,3.660644e+10,3.245160e+09,206.5,23.824092,11.048689
1,11010540,2370.0,800.0,143.0,68.0,172.0,85.0,2.0,0.0,17.0,...,0.000000e+00,0.0,6.900190e+09,111870248.0,5.324073e+08,7.375948e+09,2.476535e+08,76.5,33.755274,9.562500
2,11010550,9116.0,1549.0,1147.0,213.0,841.0,154.0,0.0,10.0,14.0,...,5.614901e+07,0.0,4.776125e+08,189999970.0,8.978345e+08,7.587584e+08,2.295424e+07,183.5,16.992102,11.846352
3,11010560,15369.0,1617.0,1269.0,136.0,1173.0,123.0,2.0,16.0,14.0,...,2.517208e+08,0.0,9.633955e+08,284394280.0,3.962739e+08,2.456528e+09,2.872706e+08,129.5,10.521179,8.008658
4,11030740,18463.0,5314.0,1138.0,442.0,1823.0,743.0,12.0,46.0,26.0,...,1.512627e+09,910011062.0,4.049663e+09,415945610.0,4.542987e+09,2.130260e+10,3.918752e+09,592.5,28.781888,11.149793


In [10]:
columns_2022 = [col.replace('_2022', '') for col in df_2023.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2023.columns = columns_2024
df_2023.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', '네일숍', '미용실', '분식전문점', '애완동물', '한식음식점', '양식음식점', '일식음식점',
       '중식음식점', '치킨전문점', '커피-음료', '패스트푸드점', '제과점', '편의점', '호프-간이주점', '지가변동률',
       '지가', '사업체수', '종사자수', '금융기관', '유통업체수', '대형마트', '쇼핑업체수', '대학교', '기타대학',
       '면적', '월평균소득', '총상주인구', '20대상주인구', '총직장인구', '20대직장인구', '슈퍼마켓',
       '20대매출건수_네일숍', '20대매출건수_미용실', '20대매출건수_분식전문점', '20대매출건수_애완동물',
       '20대매출건수_양식음식점', '20대매출건수_일식음식점', '20대매출건수_제과점', '20대매출건수_중식음식점',
       '20대매출건수_치킨전문점', '20대매출건수_커피-음료', '20대매출건수_패스트푸드점', '20대매출건수_편의점',
       '20대매출건수_한식음식점', '20대매출건수_호프-간이주점', '20대매출금액_네일숍', '20대매출금액_미용실',
       '20대매출금액_분식전문점', '20대매출금액_애완동물', '20대매출금액_양식음식점', '20대매출금액_일식음식점',
       '20대매출금액_제과점', '20대매출금액_중식음식점', '20대매출금액_치킨전문점', '20대매출금액_커피-음료',
       '20대매출금액_패스트푸드점', '20대매출금액_편의점', '20대매출금액_한식음식점', '20대매출금액_호프-간이주점',
       '총매출건수_네일숍', '총매출건수_미용실', '총매출건수_분식전문점', '총매출건수_애완동물', '총매출건수_양식음식점',
       '총매출건수_일식음식점', '총매출건수_제과

In [11]:
reg_2023 = df_2023[reg_col]
reg_2023

,ADM_CD,solo_all_ratio,solo_20s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,일식음식점,...,종사자수,금융기관,대학교,기타대학,면적,월평균소득,총상주인구,20대상주인구,총직장인구,20대직장인구
0,11010530,23.824092,11.048689,13.0,32.0,8.0,124.0,414.0,182.0,58.0,...,56046,21,0.0,0.0,1.23,5007768.0,8486.0,786.0,47822.0,6303.0
1,11010540,33.755274,9.562500,2.0,0.0,0.0,17.0,73.0,85.0,7.0,...,5118,3,0.0,0.0,1.49,3442534.0,2597.0,258.0,5862.0,726.0
2,11010550,16.992102,11.846352,0.0,10.0,7.0,14.0,38.0,37.0,5.0,...,4140,1,1.0,0.0,2.27,3647449.0,9744.0,1267.0,4115.0,695.0
3,11010560,10.521179,8.008658,2.0,16.0,3.0,14.0,55.0,18.0,6.0,...,4294,2,0.0,0.0,8.87,4504006.0,17852.0,2297.0,23179.0,2878.0
4,11030740,28.781888,11.149793,12.0,46.0,12.0,26.0,202.0,217.0,47.0,...,20440,11,0.0,0.0,3.01,4381088.0,19536.0,1460.0,41934.0,6886.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,28.862095,9.442994,0.0,8.0,1.0,20.0,67.0,44.0,7.0,...,15449,4,0.0,0.0,0.54,3479766.0,4350.0,470.0,11728.0,1263.0
418,11010610,51.245969,12.514302,10.0,34.0,13.0,238.0,1193.0,316.0,116.0,...,106826,41,1.0,0.0,2.35,3245251.0,7051.0,574.0,97451.0,19755.0
419,11220630,14.020878,11.360719,55.0,234.0,63.0,93.0,512.0,118.0,71.0,...,16795,0,0.0,0.0,1.93,3864301.0,20416.0,2439.0,16805.0,2831.0
420,11090700,23.520569,13.645079,55.0,308.0,69.0,107.0,631.0,26.0,44.0,...,7190,0,0.0,1.0,0.93,2318870.0,22868.0,2521.0,1850.0,199.0


In [12]:
df_2024 = temp_2024.copy()
df_2024.head()

,ADM_CD,total_all_2024,solo_all_2024,total_20_2024,solo_20_2024,total_25_2024,solo_25_2024,네일숍_2024,미용실_2024,분식전문점_2024,...,총매출금액_중식음식점_2023,총매출금액_치킨전문점_2023,총매출금액_커피-음료_2023,총매출금액_패스트푸드점_2023,총매출금액_편의점_2023,총매출금액_한식음식점_2023,총매출금액_호프-간이주점_2023,solo_20s_2024,solo_all_ratio,solo_20s_ratio
0,11010530,8934.0,2200.0,569.0,182.0,705.0,252.0,14.0,35.0,122.0,...,2.749773e+09,673323360.0,1.187346e+10,902523398.0,5.084405e+09,3.660644e+10,3.245160e+09,217.0,24.625028,9.863636
1,11010540,2579.0,911.0,311.0,152.0,176.0,88.0,1.0,0.0,17.0,...,0.000000e+00,0.0,6.900190e+09,111870248.0,5.324073e+08,7.375948e+09,2.476535e+08,120.0,35.323769,13.172338
2,11010550,9226.0,1582.0,1119.0,204.0,865.0,161.0,0.0,10.0,13.0,...,5.614901e+07,0.0,4.776125e+08,189999970.0,8.978345e+08,7.587584e+08,2.295424e+07,182.5,17.147193,11.536030
3,11010560,15667.0,1646.0,1238.0,131.0,1207.0,128.0,3.0,15.0,13.0,...,2.517208e+08,0.0,9.633955e+08,284394280.0,3.962739e+08,2.456528e+09,2.872706e+08,129.5,10.506159,7.867558
4,11030740,19241.0,5509.0,1111.0,439.0,1882.0,760.0,14.0,48.0,24.0,...,1.512627e+09,910011062.0,4.049663e+09,415945610.0,4.542987e+09,2.130260e+10,3.918752e+09,599.5,28.631568,10.882193


In [13]:
columns_2022 = [col.replace('_2022', '') for col in df_2024.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2024.columns = columns_2024
df_2024.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', '네일숍', '미용실', '분식전문점', '애완동물', '한식음식점', '양식음식점', '일식음식점',
       '중식음식점', '치킨전문점', '커피-음료', '패스트푸드점', '제과점', '편의점', '호프-간이주점', '지가변동률',
       '지가', '사업체수', '종사자수', '금융기관', '유통업체수', '대형마트', '쇼핑업체수', '대학교', '기타대학',
       '면적', '월평균소득', '총상주인구', '20대상주인구', '총직장인구', '20대직장인구', '슈퍼마켓',
       '20대매출건수_네일숍', '20대매출건수_미용실', '20대매출건수_분식전문점', '20대매출건수_애완동물',
       '20대매출건수_양식음식점', '20대매출건수_일식음식점', '20대매출건수_제과점', '20대매출건수_중식음식점',
       '20대매출건수_치킨전문점', '20대매출건수_커피-음료', '20대매출건수_패스트푸드점', '20대매출건수_편의점',
       '20대매출건수_한식음식점', '20대매출건수_호프-간이주점', '20대매출금액_네일숍', '20대매출금액_미용실',
       '20대매출금액_분식전문점', '20대매출금액_애완동물', '20대매출금액_양식음식점', '20대매출금액_일식음식점',
       '20대매출금액_제과점', '20대매출금액_중식음식점', '20대매출금액_치킨전문점', '20대매출금액_커피-음료',
       '20대매출금액_패스트푸드점', '20대매출금액_편의점', '20대매출금액_한식음식점', '20대매출금액_호프-간이주점',
       '총매출건수_네일숍', '총매출건수_미용실', '총매출건수_분식전문점', '총매출건수_애완동물', '총매출건수_양식음식점',
       '총매출건수_일식음식점', '총매출건수_제과

In [14]:
reg_2024 = df_2024[reg_col]
reg_2024

,ADM_CD,solo_all_ratio,solo_20s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,일식음식점,...,종사자수,금융기관,대학교,기타대학,면적,월평균소득,총상주인구,20대상주인구,총직장인구,20대직장인구
0,11010530,24.625028,9.863636,14.0,35.0,10.0,122.0,407.0,187.0,63.0,...,56046,21,0.0,0.0,1.23,5007768.0,8486.0,786.0,47822.0,6303.0
1,11010540,35.323769,13.172338,1.0,0.0,1.0,17.0,76.0,85.0,7.0,...,5118,3,0.0,0.0,1.49,3442534.0,2597.0,258.0,5862.0,726.0
2,11010550,17.147193,11.536030,0.0,10.0,8.0,13.0,36.0,38.0,5.0,...,4140,1,1.0,0.0,2.27,3647449.0,9744.0,1267.0,4115.0,695.0
3,11010560,10.506159,7.867558,3.0,15.0,4.0,13.0,53.0,20.0,7.0,...,4294,2,0.0,0.0,8.87,4504006.0,17852.0,2297.0,23179.0,2878.0
4,11030740,28.631568,10.882193,14.0,48.0,13.0,24.0,199.0,236.0,48.0,...,20440,11,0.0,0.0,3.01,4381088.0,19536.0,1460.0,41934.0,6886.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,29.180084,8.870968,0.0,8.0,1.0,19.0,67.0,43.0,7.0,...,15449,4,0.0,0.0,0.54,3479766.0,4350.0,470.0,11728.0,1263.0
418,11010610,48.905192,11.020078,12.0,32.0,20.0,235.0,1219.0,320.0,116.0,...,106826,41,1.0,0.0,2.35,3245251.0,7051.0,574.0,97451.0,19755.0
419,11220630,13.962063,10.738148,61.0,230.0,61.0,85.0,524.0,118.0,72.0,...,16795,0,0.0,0.0,1.93,3864301.0,20416.0,2439.0,16805.0,2831.0
420,11090700,25.509848,12.617279,62.0,304.0,73.0,93.0,601.0,23.0,43.0,...,7190,0,0.0,1.0,0.93,2318870.0,22868.0,2521.0,1850.0,199.0


# 데이터 정규화

In [15]:
### 데이터 정규화

year_list = ['2022', '2023', '2024']

for year in year_list:
    globals()[f'reg_{year}'] = globals()[f'reg_{year}'].replace('-', '0')

    std_scaler = StandardScaler()
    df_std = std_scaler.fit_transform(globals()[f'reg_{year}'])
    df_std = pd.DataFrame(df_std)
    df_std.columns = globals()[f'reg_{year}'].columns
    df_std.drop(columns = 'ADM_CD', inplace = True)

    globals()[f'reg_{year}_temp'] = df_std.copy()

reg_2022 = reg_2022_temp.copy()
reg_2023 = reg_2023_temp.copy()
reg_2024 = reg_2024_temp.copy()


In [16]:
reg_2022

,solo_all_ratio,solo_20s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,일식음식점,중식음식점,...,종사자수,금융기관,대학교,기타대학,면적,월평균소득,총상주인구,20대상주인구,총직장인구,20대직장인구
0,0.415566,0.085282,-0.677852,-1.045196,-0.925207,0.940577,0.113738,1.379592,0.331708,-0.080635,...,1.957975,4.939764,-0.288565,-0.223050,-0.121303,1.696731,-1.460915,-1.366613,1.621306,0.989573
1,1.502040,-0.042339,-1.164200,-1.293013,-1.350150,-0.939767,-1.030905,0.246323,-0.863220,-0.777185,...,-0.393207,0.450037,-0.288565,-0.223050,0.043764,0.058449,-2.222310,-1.935910,-0.243128,-0.283729
2,-0.245995,0.208340,-1.204729,-1.210407,-1.095184,-0.991049,-1.138636,-0.308747,-0.937903,-0.802983,...,-0.438358,0.001064,2.543395,-0.223050,0.538966,0.272927,-1.406648,-1.303358,-0.320753,-0.290806
3,-0.859866,-0.383977,-1.123671,-1.160844,-1.222667,-0.973955,-1.088137,-0.516898,-0.913008,-0.802983,...,-0.431248,0.225550,-0.288565,-0.223050,4.729134,1.169459,-0.481015,-0.614993,0.526329,0.207601
4,0.913616,0.129359,-0.758910,-0.913027,-0.755230,-0.768827,-0.613447,1.691819,0.032976,-0.287020,...,0.314161,2.245928,-0.288565,-0.223050,1.008773,1.040804,-0.175860,-0.409104,1.359681,1.122679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,0.851739,-0.240419,-1.204729,-1.226928,-1.307656,-0.871391,-1.051104,-0.227799,-0.863220,-0.828781,...,0.083742,0.674523,-0.288565,-0.223050,-0.559366,0.097418,-2.035526,-1.828624,0.017520,-0.161125
418,2.882073,0.275783,-0.758910,-1.003893,-0.882713,2.872204,2.672352,2.917600,1.651107,0.796503,...,4.302325,9.205005,2.543395,-0.223050,0.589756,-0.148041,-1.713129,-1.589867,3.826500,4.060838
419,-0.522344,0.153762,0.943307,0.582135,1.157012,0.530320,0.490797,0.477603,0.680229,0.280540,...,0.145883,-0.223422,-0.288565,-0.223050,0.323109,0.499899,-0.275774,-0.360112,0.243109,0.196870
420,0.491312,0.403592,0.902778,1.259501,1.242001,0.684167,0.925088,-0.389695,0.207236,0.125751,...,-0.297549,-0.223422,-0.288565,4.483302,-0.311765,-1.117656,-0.058043,0.264377,-0.421395,-0.404050


In [17]:
selected_var = reg_2022.columns[2:]
selected_var

Index(['네일숍', '미용실', '애완동물', '분식전문점', '한식음식점', '양식음식점', '일식음식점', '중식음식점',
       '치킨전문점', '커피-음료', '패스트푸드점', '제과점', '호프-간이주점', '유통업체수', '대형마트', '쇼핑업체수',
       '편의점', '슈퍼마켓', '지가변동률', '지가', '사업체수', '종사자수', '금융기관', '대학교', '기타대학',
       '면적', '월평균소득', '총상주인구', '20대상주인구', '총직장인구', '20대직장인구'],
      dtype='object')

In [18]:
### 분석 - OLS

depend_list = ['solo_all_ratio', 'solo_20s_ratio']
depend_name = ['all', '20']
selected_var = reg_2022.columns[2:]

year_list = ['2022', '2023', '2024']

df_ols = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}']
    for idx, depend in enumerate(depend_list):
        X = sm.add_constant(df[selected_var])

        # Fit the OLS model
        model = sm.OLS(df[depend], X)
        result = model.fit()

        summary = result.summary()

        temp = summary.tables[1].as_csv()

        ols_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        ols_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']
        ols_result = ols_result[['var', 'coef', 'p']]
        ols_result['p'] = ols_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        ols_result['coef'] = ols_result['coef'].apply(lambda x: round(x, 3))
        ols_result.columns = [f'var', f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']
        ols_result['var'] = ols_result['var'].str.strip()

        globals()[f'ols_{year}_{depend}'] = ols_result

        if (year == '2022') & (idx == 0):
            df_ols = ols_result
        else:
            df_ols = df_ols.merge(ols_result,
                                        how = 'left',
                                        on = 'var')

In [19]:
ols_2022_solo_20s_ratio

,var,c_2022_20,p_2022_20
0,const,0.000,
1,네일숍,-0.043,
2,미용실,-0.334,+
3,애완동물,-0.190,
4,분식전문점,-0.070,
5,한식음식점,0.234,
6,양식음식점,0.095,
7,일식음식점,-0.092,
8,중식음식점,0.021,
9,치킨전문점,0.115,


In [20]:
df_ols.to_csv(f'{final_result}/ols_all.csv', index = False)
df_ols.to_excel(f'{final_result}/ols_all.xlsx', index = False)

## 컬럼명을 영어로 바꾸는 준비
- 전진선택법, 후진제거법은 변수명 영어로

In [21]:
reg_2022_en = reg_2022.copy()

new_columns = ['var{}'.format(i) for i in range(1,32)]
reg_2022_en.columns = depend_list + new_columns

In [22]:
reg_2022_en

,solo_all_ratio,solo_20s_ratio,var1,var2,var3,var4,var5,var6,var7,var8,...,var22,var23,var24,var25,var26,var27,var28,var29,var30,var31
0,0.415566,0.085282,-0.677852,-1.045196,-0.925207,0.940577,0.113738,1.379592,0.331708,-0.080635,...,1.957975,4.939764,-0.288565,-0.223050,-0.121303,1.696731,-1.460915,-1.366613,1.621306,0.989573
1,1.502040,-0.042339,-1.164200,-1.293013,-1.350150,-0.939767,-1.030905,0.246323,-0.863220,-0.777185,...,-0.393207,0.450037,-0.288565,-0.223050,0.043764,0.058449,-2.222310,-1.935910,-0.243128,-0.283729
2,-0.245995,0.208340,-1.204729,-1.210407,-1.095184,-0.991049,-1.138636,-0.308747,-0.937903,-0.802983,...,-0.438358,0.001064,2.543395,-0.223050,0.538966,0.272927,-1.406648,-1.303358,-0.320753,-0.290806
3,-0.859866,-0.383977,-1.123671,-1.160844,-1.222667,-0.973955,-1.088137,-0.516898,-0.913008,-0.802983,...,-0.431248,0.225550,-0.288565,-0.223050,4.729134,1.169459,-0.481015,-0.614993,0.526329,0.207601
4,0.913616,0.129359,-0.758910,-0.913027,-0.755230,-0.768827,-0.613447,1.691819,0.032976,-0.287020,...,0.314161,2.245928,-0.288565,-0.223050,1.008773,1.040804,-0.175860,-0.409104,1.359681,1.122679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,0.851739,-0.240419,-1.204729,-1.226928,-1.307656,-0.871391,-1.051104,-0.227799,-0.863220,-0.828781,...,0.083742,0.674523,-0.288565,-0.223050,-0.559366,0.097418,-2.035526,-1.828624,0.017520,-0.161125
418,2.882073,0.275783,-0.758910,-1.003893,-0.882713,2.872204,2.672352,2.917600,1.651107,0.796503,...,4.302325,9.205005,2.543395,-0.223050,0.589756,-0.148041,-1.713129,-1.589867,3.826500,4.060838
419,-0.522344,0.153762,0.943307,0.582135,1.157012,0.530320,0.490797,0.477603,0.680229,0.280540,...,0.145883,-0.223422,-0.288565,-0.223050,0.323109,0.499899,-0.275774,-0.360112,0.243109,0.196870
420,0.491312,0.403592,0.902778,1.259501,1.242001,0.684167,0.925088,-0.389695,0.207236,0.125751,...,-0.297549,-0.223422,-0.288565,4.483302,-0.311765,-1.117656,-0.058043,0.264377,-0.421395,-0.404050


In [23]:
col_dict = {}

for i in range(len(df_std.columns)):
    col_dict[reg_2022_en.columns[i]] = reg_2022.columns[i]
    col_dict[reg_2022.columns[i]] = reg_2022_en.columns[i]

col_dict.update({'Intercept' : '상수',
                 '상수' : 'intercept',
                 'const' : '상수',
                 '상수' : 'const'})

# 전진선택법

In [24]:
### 전진선택법 함수

def forward_selection(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {}".format(response, ' + '.join(selected + [candidate]))
            score = sm.OLS.from_formula(formula, data).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
        else:
            break
    formula = "{} ~ {}".format(response, ' + '.join(selected))
    model = sm.OLS.from_formula(formula, data).fit()
    return model


In [25]:
### 분석 - fwd

depend_list = ['solo_all_ratio', 'solo_20s_ratio']
depend_name = ['all', '20']
selected_var = reg_2022.columns[2:]

year_list = ['2022', '2023', '2024']

df_fwd = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}'].copy()

    new_columns = ['var{}'.format(i) for i in range(1,32)]
    df.columns = depend_list + new_columns


    for idx, depend in enumerate(depend_list):
        drop_list = [item for item in depend_list if item != depend]

        model = forward_selection(df.drop(columns = drop_list), depend)
        summary = model.summary()
        temp = summary.tables[1].as_csv()

        fwd_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        fwd_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']

        fwd_result['var'] = fwd_result['var'].str.strip()
        fwd_result['coef'] = fwd_result['coef'].apply(lambda x: round(x, 3))
        fwd_result['var_mapped'] = fwd_result['var'].apply(lambda x: col_dict[x])
        fwd_result = fwd_result[['var', 'var_mapped', 'coef', 'p' ]]
        fwd_result['var_mapped'] = fwd_result['var_mapped'].str.strip()
        fwd_result['p'] = fwd_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        fwd_result.columns = [f'var2', f'var',f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']
        globals()[f'fwd_{year}_{depend}'] = fwd_result

        if (year == '2022') & (idx == 0):
            df_fwd = fwd_result
        else:
            df_fwd = df_fwd.merge(fwd_result,
                                        how = 'left',
                                        on = ['var', 'var2'])

In [26]:
df_fwd.to_csv(f'{final_result}/fwd_all.csv', index = False)
df_fwd.to_excel(f'{final_result}/fwd_all.xlsx', index = False)

In [27]:
df_fwd

,var2,var,c_2022_all,p_2022_all,c_2022_20,p_2022_20,c_2023_all,p_2023_all,c_2023_20,p_2023_20,c_2024_all,p_2024_all,c_2024_20,p_2024_20
0,Intercept,상수,0.000,,0.000,,-0.000,,0.000,,0.000,,-0.000,
1,var21,사업체수,0.166,+,-0.131,+,0.372,+,0.148,+,0.346,+,0.113,
2,var27,월평균소득,-0.227,+,0.074,+,-0.343,+,-0.086,,-0.394,+,-0.082,
3,var28,총상주인구,-0.993,+,-0.987,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,var29,20대상주인구,0.993,+,1.251,+,-0.286,+,-0.097,+,-0.292,+,-0.091,+
5,var23,금융기관,0.181,+,NaN,NaN,0.085,+,-0.065,,0.069,,-0.095,+
6,var6,양식음식점,0.093,+,NaN,NaN,0.114,+,0.101,,0.113,+,0.116,
7,var2,미용실,-0.274,+,-0.265,+,NaN,NaN,NaN,NaN,-0.269,,NaN,NaN
8,var4,분식전문점,0.195,+,NaN,NaN,0.520,+,NaN,NaN,0.558,+,NaN,NaN
9,var30,총직장인구,-0.580,+,NaN,NaN,-0.617,+,NaN,NaN,-0.673,+,NaN,NaN


### 후진제거법

In [28]:
def backward_elimination(data, response, significance_level=0.05):
    features = data.columns.tolist()
    features.remove(response)

    while len(features) > 0:
        features_with_constant = sm.add_constant(data[features])  # Add a constant term for the intercept
        model = sm.OLS(data[response], features_with_constant).fit()
        p_values = model.pvalues.iloc[1:]  # Get p-values for the features only
        max_p_value = p_values.max()  # Find the feature with the maximum p-value
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break

    model = sm.OLS(data[response], sm.add_constant(data[features])).fit()
    return model


In [29]:
### 분석 - bwd

depend_list = ['solo_all_ratio', 'solo_20s_ratio']
depend_name = ['all', '20']
selected_var = reg_2022.columns[2:]

year_list = ['2022', '2023', '2024']

df_bwd = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}'].copy()

    new_columns = ['var{}'.format(i) for i in range(1,32)]
    df.columns = depend_list + new_columns


    for idx, depend in enumerate(depend_list):
        drop_list = [item for item in depend_list if item != depend]

        model = backward_elimination(df.drop(columns = drop_list), depend)
        summary = model.summary()
        temp = summary.tables[1].as_csv()

        bwd_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        bwd_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']

        bwd_result['var'] = bwd_result['var'].str.strip()
        bwd_result['coef'] = bwd_result['coef'].apply(lambda x: round(x, 3))
        bwd_result['var_mapped'] = bwd_result['var'].apply(lambda x: col_dict[x])
        bwd_result = bwd_result[['var', 'var_mapped', 'coef', 'p' ]]
        bwd_result['var_mapped'] = bwd_result['var_mapped'].str.strip()
        bwd_result['p'] = bwd_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        bwd_result.columns = [f'var2', f'var',f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']

        globals()[f'bwd_{year}_{depend}'] = bwd_result

        if (year == '2022') & (idx == 0):
            df_bwd = bwd_result
        else:
            df_bwd = df_bwd.merge(bwd_result,
                                        how = 'left',
                                       on = ['var', 'var2'])

In [30]:
df_bwd.to_csv(f'{final_result}/bwd_all.csv', index = False)
df_bwd.to_excel(f'{final_result}/bwd_all.xlsx', index = False)

In [31]:
df_bwd

,var2,var,c_2022_all,p_2022_all,c_2022_20,p_2022_20,c_2023_all,p_2023_all,c_2023_20,p_2023_20,c_2024_all,p_2024_all,c_2024_20,p_2024_20
0,const,상수,0.000,,0.000,,-0.000,,0.000,,0.000,,-0.000,
1,var2,미용실,-0.280,+,-0.278,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,var4,분식전문점,0.194,+,NaN,NaN,0.492,+,NaN,NaN,0.537,+,NaN,NaN
3,var6,양식음식점,0.097,+,NaN,NaN,0.129,+,NaN,NaN,0.134,+,NaN,NaN
4,var21,사업체수,0.167,+,-0.130,+,0.376,+,0.143,+,0.360,+,0.117,+
5,var23,금융기관,0.181,+,NaN,NaN,0.092,+,NaN,NaN,NaN,NaN,-0.096,+
6,var27,월평균소득,-0.228,+,0.084,+,-0.344,+,NaN,NaN,-0.370,+,NaN,NaN
7,var28,총상주인구,-0.976,+,-0.997,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,var29,20대상주인구,0.989,+,1.258,+,-0.281,+,-0.085,+,-0.301,+,-0.113,+
9,var30,총직장인구,-0.560,+,NaN,NaN,-0.617,+,-0.229,+,-0.700,+,NaN,NaN


# 각연도별 통합

In [32]:
depend_list = ['solo_all_ratio', 'solo_20s_ratio']
depend_name = ['all', '20']
selected_var = reg_2022.columns[2:]

year_list = ['2022', '2023', '2024']

df_bwd = pd.DataFrame()

for year in year_list:

    for idx, depend in enumerate(depend_list):

        ols = globals()[f'ols_{year}_{depend}']
        fwd = globals()[f'fwd_{year}_{depend}']
        bwd = globals()[f'bwd_{year}_{depend}']

        ols['var'] = ols['var'].apply(lambda x: x if x != 'const' else '상수')

        temp = ols.merge(fwd.drop(columns = 'var2'),
                    how = 'outer',
                    on = 'var',
                    suffixes=('_ols', '_fwd'))
        temp = temp.merge(bwd.drop(columns = 'var2'),
                    how = 'outer',
                    on = 'var',
                    suffixes=('_fwd', '_bwd'))

        temp = temp.sort_values(by=[f'c_{year}_{depend_name[idx]}_fwd', f'p_{year}_{depend_name[idx]}'])

        globals()[f'result_{year}_{depend}']= temp

        globals()[f'result_{year}_{depend}'].to_csv(f'{final_result}/{year}_{depend}.csv', index = False)
        globals()[f'result_{year}_{depend}'].to_excel(f'{final_result}/{year}_{depend}.xlsx', index = False)

        globals()[f'result_{year}_{idx}'] = temp

In [33]:
result_2022_0

,var,c_2022_all_ols,p_2022_all_ols,c_2022_all_fwd,p_2022_all_fwd,c_2022_all,p_2022_all
28,총상주인구,-1.001,+,-0.993,+,-0.976,+
30,총직장인구,-0.591,+,-0.580,+,-0.560,+
2,미용실,-0.346,+,-0.274,+,-0.280,+
27,월평균소득,-0.229,+,-0.227,+,-0.228,+
0,상수,0.000,,0.000,,0.000,
26,면적,0.049,,0.044,,NaN,NaN
6,양식음식점,0.109,+,0.093,+,0.097,+
21,사업체수,0.208,+,0.166,+,0.167,+
23,금융기관,0.185,+,0.181,+,0.181,+
4,분식전문점,0.228,+,0.195,+,0.194,+
